In [5]:
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
import torch.nn.functional as F
from nltk.tokenize import word_tokenize
import pickle

import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

import model as m
from utils import pad_sents, data_iterator

In [6]:
if torch.cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"
device = torch.device(dev)

In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/albertxu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [33]:
# df = pd.read_csv('/content/drive/My Drive/577-project/reason-dataset.csv')
df = pickle.load(open('reasons_complete_df.pkl', 'rb'))
df['lower_line'] = df['line'].apply(lambda r: r.lower()).apply(word_tokenize)

In [9]:
CoPAs = pd.read_csv('principle_argument_CoPA/PA_list.txt', header=None, sep='$', squeeze=True)
copas_t = CoPAs.apply(lambda row: row.lower()).apply(word_tokenize)

copa_w2v = pickle.load(open('principle_argument_CoPA/PA_embeds_avg_w2v.pkl', 'rb'))
copa_tfidf = pickle.load(open('principle_argument_CoPA/PA_embeds_tfidf_df.pkl', 'rb'))

In [78]:
copa_tfidf['CoPA_embedding'][0].shape

(300,)

In [75]:
df.head()

,id,topic,stance,substance,source,line,claim_tf_idf_embedding,tf_idf_pa_similarity,claim_SBERT_embedding,lower_line
0,0,abortion,Con,c-adopt,Q34,And if it is not possible for your to have a b...,"[227.53242, 95.16571, 90.81468, 413.58224, -44...","[0.48416767, 0.4241976, 0.54075974, 0.69915974...","[0.021533493, -0.16520593, 0.35402358, 0.43329...","[and, if, it, is, not, possible, for, your, to..."
1,1,abortion,Con,c-kill,Q34,I believe that abortion cannot be justified be...,"[32.17483, 92.68718, 283.92633, 117.998245, -2...","[0.47884166, 0.43939564, 0.5063248, 0.70720005...","[0.47159326, 0.4925952, -0.23349553, -0.087148...","[i, believe, that, abortion, can, not, be, jus..."
2,2,abortion,Con,c-baby_right,Q34,"Moreover, United Nations declaration says chil...","[-27.021324, -21.564283, 46.234787, 301.42996,...","[0.43071118, 0.45206025, 0.48450536, 0.5497553...","[0.4615807, 0.68479395, 0.737707, 0.26567188, ...","[moreover, ,, united, nations, declaration, sa..."
3,3,abortion,Con,c-baby_right,M25,"Even if it doesn't have a brain, my belief is ...","[357.5555, -13.070822, 262.6633, 333.12784, -2...","[0.44407952, 0.4239362, 0.49705744, 0.70402575...","[0.78423434, 0.56576806, 0.69618547, 0.0808582...","[even, if, it, does, n't, have, a, brain, ,, m..."
4,4,abortion,Con,c-sex,O43,"Can't you use a condom while having sex. Yes, ...","[182.9457, 77.004105, 34.06093, 160.79187, -15...","[0.41161808, 0.39982748, 0.43759474, 0.6643834...","[-0.19426306, 0.5660061, -0.13704692, -0.07844...","[ca, n't, you, use, a, condom, while, having, ..."


In [53]:
import importlib
importlib.reload(m)

<module 'model' from '/Users/albertxu/577-Project/model.py'>

In [54]:
tr = m.MatmulT(100, dropout=.3)
#tr = m.LinearT(100, len(copas), dropout=.3)
model = m.Model(5, 10, 2, tr, dropout=.3, embed=False)

In [58]:
z = model(torch.tensor([[1,1,2,3,1],[2,2,2,3,1]], dtype=float), torch.tensor([[1,1,2,3,1],[2,2,2,3,1]]))

In [74]:
(z/torch.norm(z, dim=1, keepdim=True))

tensor([[0.2500, 0.2500, 0.5000, 0.7500, 0.2500],
        [0.4264, 0.4264, 0.4264, 0.6396, 0.2132]], dtype=torch.float64)

In [30]:
z = model.embedding(torch.tensor([[1,1,2,3,1],[2,2,2,3,1]]))
model.encoder(z)

(tensor([[[-0.1403, -0.1304, -0.0560, -0.1456, -0.0682,  0.0268,  0.1162,
            0.0770, -0.1151,  0.1242, -0.1999, -0.1103, -0.3527,  0.1362,
           -0.2672,  0.1425,  0.0383,  0.0856, -0.0584,  0.1248],
          [-0.1756, -0.1810, -0.0847, -0.1967, -0.1038,  0.0636,  0.1584,
            0.1186, -0.1596,  0.1921, -0.1815, -0.1360, -0.3064,  0.1273,
           -0.2501,  0.1218,  0.0742,  0.0742, -0.0705,  0.1296],
          [-0.0999, -0.1565, -0.0822,  0.0034, -0.1131, -0.0632,  0.2364,
            0.1921, -0.2560, -0.0390, -0.1688, -0.3564, -0.1007,  0.1872,
           -0.1550,  0.1140,  0.1996,  0.0561, -0.1113,  0.0832],
          [-0.0849, -0.0482,  0.1167,  0.1129,  0.1532, -0.1485,  0.1242,
            0.0247, -0.0558, -0.1259, -0.1755, -0.0476, -0.0557, -0.0165,
            0.1652,  0.1688,  0.0959,  0.0956, -0.0610,  0.3632],
          [-0.1661, -0.1376,  0.0195, -0.0819, -0.0258, -0.0465,  0.1400,
            0.1331, -0.1401, -0.0166, -0.1386, -0.0523, -0.2035,  0.03

In [28]:
z

tensor([[-1.0701,  0.4017, -1.2252,  1.1666,  0.8044, -0.6030, -1.6613,  0.0216,
         -1.6670,  0.8180],
        [-1.0701,  0.4017, -1.2252,  1.1666,  0.8044, -0.6030, -1.6613,  0.0216,
         -1.6670,  0.8180],
        [-0.7233, -1.8268, -1.2373,  0.8108,  1.9194, -1.7749, -0.1621, -0.1469,
         -0.4632, -0.2646],
        [ 0.6387,  0.4257, -0.7885,  0.2793, -0.9167,  1.5537,  0.8152, -1.3804,
          0.6980, -0.1623],
        [-1.0701,  0.4017, -1.2252,  1.1666,  0.8044, -0.6030, -1.6613,  0.0216,
         -1.6670,  0.8180]], grad_fn=<EmbeddingBackward>)

In [6]:
vocab = set()
for r in df['lower_line']:
    vocab.update(r)
for r in copas_t:
    vocab.update(r)
len(vocab)

6182

In [7]:
word2ix = dict(zip(vocab, range(1, len(vocab)+1)))
word2ix['<PAD>']= 0

In [8]:
text = df['lower_line'].apply(lambda r: [word2ix[w] for w in r]).tolist()
copas = copas_t.apply(lambda r: [word2ix[w] for w in r]).tolist()
stances = (df['stance'] == 'Pro').astype(int).tolist()

In [9]:
model = m.Model1(len(vocab) + 1, 100, 2)
optimizer = optim.Adam(model.parameters(), weight_decay=.01)


/Users/albertxu/.pyenv/versions/3.8.1/envs/pytorch/lib/python3.8/site-packages/torch/nn/modules/rnn.py:47: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [33]:
tensor_copas = torch.tensor(pad_sents(copas))
n_iter = 50
for it in range(n_iter):
    # for claim, stance in data_iterator(corrected, stances):
    model.train()
    for claim, stance in data_iterator(text, stances):
        input_vec = torch.tensor(pad_sents(claim))
        stance_vec = torch.tensor(stance)
        probs = model(input_vec.to(device), tensor_copas.to(device))
        loss = F.nll_loss(probs, stance_vec.to(device))
        loss.backward()
        optimizer.step()
    
    print(f'epoch {it} done.')
    model.eval()
    with torch.no_grad():
        input_vec = torch.tensor(pad_sents(text))
        stance_vec = torch.tensor(stances)
        probs = model(input_vec.to(device), tensor_copas.to(device))
        l = F.nll_loss(probs, stance_vec.to(device)).item()
        print(f'Test loss: {l}')
        
        nwrong = np.count_nonzero(np.argmax(probs.cpu().numpy(), 1) - stances)
        print(f'Test %wrong: {nwrong / len(stances)}')


epoch 0 done.
Test loss: 0.6637189388275146
Test acc: 0.3648174157303371
epoch 1 done.
Test loss: 0.6890787482261658
Test acc: 0.3648174157303371
epoch 2 done.
Test loss: 0.7924987077713013
Test acc: 0.6351825842696629
epoch 3 done.
Test loss: 0.9858576059341431
Test acc: 0.3648174157303371
epoch 4 done.
Test loss: 0.9146988987922668
Test acc: 0.6351825842696629
epoch 5 done.
Test loss: 0.7089576125144958
Test acc: 0.3648174157303371
epoch 6 done.
Test loss: 0.6765966415405273
Test acc: 0.3648174157303371
epoch 7 done.
Test loss: 0.6806909441947937
Test acc: 0.367626404494382
epoch 8 done.
Test loss: 0.7165311574935913
Test acc: 0.3648174157303371
epoch 9 done.
Test loss: 0.7917943596839905
Test acc: 0.6351825842696629
epoch 10 done.
Test loss: 0.7002094388008118
Test acc: 0.3648174157303371
epoch 11 done.
Test loss: 0.8766077160835266
Test acc: 0.3648174157303371
epoch 12 done.
Test loss: 1.1894946098327637
Test acc: 0.6351825842696629
epoch 13 done.
Test loss: 2.525092363357544
Test 

KeyboardInterrupt: 